In [3]:
import pandas as pd
df = pd.read_csv('processed_lyrics.csv')


In [4]:
subset = df.sample(n=1000)

In [5]:
subset

,ALink,SName,SLink,Lyric,Idiom,Genre,Artist
60661,/kanye-west/,Silver Surfer Intermission,/kanye-west/silver-surfer-intermission.html,"Yeezy, Yeezy, what's good?. It's your boy Max ...",ENGLISH,Hip Hop,Kanye West
82455,/katy-perry/,E.T. (feat. Kanye West),/katy-perry/e-t-feat-kanye-west.html,(Kanye West). I got a dirty mind. I got filthy...,ENGLISH,Pop,Katy Perry
63096,/omarion/,Sex Playlist,/omarion/sex-playlist.html,Can I believe I got it all alone?. Can I belie...,ENGLISH,Hip Hop,Omarion
75386,/cyndi-lauper/,Hard Candy Christmas (Feat. Alison Krauss),/cyndi-lauper/hard-candy-christmas-feat-alison...,"Hey, maybe I'll dye my hair. Maybe I'll move s...",ENGLISH,Pop,Cyndi Lauper
22413,/korn/,Dead Bodies Everywhere,/korn/dead-bodies-everywhere.html,"Come on, step inside, and you'll realize.. Tel...",ENGLISH,Rock,Korn
...,...,...,...,...,...,...,...
57268,/drake/,Ratchet Happy Birthday,/drake/ratchet-happy-birthday.html,"It's your fuckin' birthday. (oh, birthday, hap...",ENGLISH,Hip Hop,Drake
84733,/madonna/,Joan of Arc,/madonna/joan-of-arc.html,Each time they take a photograph. I lose a par...,ENGLISH,Pop,Madonna
50138,/u2/,Walk On,/u2/walk-on.html,And love is not the easy thing. The only bagga...,ENGLISH,Rock,U2
73479,/charice/,And I'm Telling You I'm Not Going,/charice/and-im-telling-you-im-not-going.html,And I am telling you. I'm not going.. You're t...,ENGLISH,Pop,Charice


In [6]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(subset, test_size=0.2, random_state=42)
X_train = train.loc[:, train.columns != 'Genre']
y_train = train['Genre']
X_test = test.loc[:, test.columns != 'Genre']
y_test = test['Genre']

In [7]:
X_train

,ALink,SName,SLink,Lyric,Idiom,Artist
26200,/mindflow/,Thousand Miles From You,/mindflow/thousand-miles-from-you.html,"I see a thousand miles through you,. From love...",ENGLISH,MindFlow
91063,/shania-twain/,Blue Eyes Crying In The Rain,/shania-twain/blue-eyes-crying-in-the-rain.html,In the twilight glow i see her. Blue eyes cryi...,ENGLISH,Shania Twain
16973,/gram-parsons/,Streets Of Baltimore,/gram-parsons/streets-of-baltimore.html,Well I sold the farm to take my woman where sh...,ENGLISH,Gram Parsons
21777,/keane/,Watch How You Go,/keane/watch-how-you-go.html,"The more we rush about, the less we do. I neve...",ENGLISH,Keane
33266,/r-e-m/,It's A Free World Baby,/r-e-m/its-a-free-world-baby.html,(Berry/Buck/Mills/Stipe). . I was hungry when ...,ENGLISH,R.E.M.
...,...,...,...,...,...,...
19280,/janis-joplin/,Gutra's Garden,/janis-joplin/gutras-garden.html,Gutra. Now now now now now now now now now now...,ENGLISH,Janis Joplin
41544,/sum-41/,88,/sum-41/88.html,Wait how long would you wait. just for me to c...,ENGLISH,Sum 41
89032,/prefab-sprout/,When The Angels,/prefab-sprout/when-the-angels.html,When the angels take the angel voive away. ome...,ENGLISH,Prefab Sprout
57520,/e-40/,The Mail Man,/e-40/the-mail-man.html,Kick this shit about the mail man. [ VERSE 1 ]...,ENGLISH,E-40


In [10]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

class PairedSentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform1(self, sentence):
        sentence = sentence.split()
        vector = np.array((sum(nlp.vocab[word].vector for word in sentence))/len(sentence))
        return (vector)

    # Vectorize a single row of the dataframe.
    def _transform2(self, row):
        # TODO: Replace the following line with your own code
        sentence = self._transform1(row.Lyric)
        return sentence

    def transform(self, X):
        return np.concatenate(
            [self._transform2(row).reshape(1, -1) for row in X.itertuples()]
        )

In [11]:
# TODO: Enter code here to implement the word embeddings classifier. Print the classification report.
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('vectorizer', PairedSentenceVectorizer()),
                       ('clf', MLPClassifier(random_state=1,  max_iter=8, hidden_layer_sizes=(300, 300, 300)))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
target_names = subset['Genre'].unique()
print(classification_report(y_test, y_pred, target_names=target_names))

/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

     Hip Hop       0.42      0.81      0.56        31
         Pop       0.61      0.17      0.27        63
        Rock       0.66      0.76      0.71       106

    accuracy                           0.58       200
   macro avg       0.56      0.58      0.51       200
weighted avg       0.61      0.58      0.55       200

